In [ ]:
import os
os.chdir("/root/pydistilledfdtd")

In [1]:
import torch
import torch.nn as nn
from src.dataset import pca_data_loader
from src.model import DistillModel, StudentLSTMModule
from tqdm import tqdm

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载模型
load_path = "../data/processed/param-1729437462.4108374.pth"
check_point = torch.load(load_path, weights_only=False)

radius = check_point["radius"]
student_model = StudentLSTMModule(input_size=10, hidden_size=128, output_size=10, num_layer=2)
student_model.load_state_dict(check_point["model_state_dict"])
model = DistillModel(radius, student_model, expand_method='sin')
model = model.to(device)

In [8]:
# 加载数据
_, test_loader = pca_data_loader(batch_size=64)

Loading PCA data from F:\pyDistilledFDTD\data\cache\pca_data_loader\pca-components-10\batch-size-64.pkl


In [9]:
class MAPELoss(nn.Module):
    def __init__(self):
        super(MAPELoss, self).__init__()

    def forward(self, target, output):
        return torch.mean(torch.abs((output - target) / target))
    
class R2Score(nn.Module):
    def __init__(self):
        super(R2Score, self).__init__()

    def forward(self, target, output):
        return 1 - torch.sum((output - target) ** 2) / torch.sum((target - torch.mean(target)) ** 2)
    
mape_loss = MAPELoss()
r2_score = R2Score()

In [10]:
# 测试模型
model.eval()
with torch.no_grad():
    with tqdm(test_loader) as tq:
        for inputs, _ in tq:
            inputs = inputs.to(device)
            fdtd_output, lstm_output = model(inputs)
            mape = mape_loss(inputs, fdtd_output)
            r2 = r2_score(inputs, fdtd_output)
            tq.set_postfix(mape=f"{mape.item():.2f}%", r2=f"{r2.item():.4f}")
            

  0%|          | 0/157 [01:05<?, ?it/s]


KeyboardInterrupt: 